In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('House_Pricing.csv')
data.head()

,ID,Date House was Sold,Sale Price,No of Bedrooms,No of Bathrooms,Flat Area (in Sqft),Lot Area (in Sqft),No of Floors,Waterfront View,No of Times Visited,...,Overall Grade,Area of the House from Basement (in Sqft),Basement Area (in Sqft),Age of House (in Years),Renovated Year,Zipcode,Latitude,Longitude,Living Area after Renovation (in Sqft),Lot Area after Renovation (in Sqft)
0,7129300520,14 October 2017,221900.0,3,1.00,1180.0,5650.0,1.0,No,NaN,...,7,1180.0,0,63,0,98178.0,47.5112,-122.257,1340.0,5650
1,6414100192,14 December 2017,538000.0,3,2.25,2570.0,7242.0,2.0,No,NaN,...,7,2170.0,400,67,1991,98125.0,47.7210,-122.319,1690.0,7639
2,5631500400,15 February 2016,180000.0,2,1.00,770.0,10000.0,1.0,No,NaN,...,6,770.0,0,85,0,98028.0,47.7379,-122.233,2720.0,8062
3,2487200875,14 December 2017,604000.0,4,3.00,1960.0,5000.0,1.0,No,NaN,...,7,1050.0,910,53,0,98136.0,47.5208,-122.393,1360.0,5000
4,1954400510,15 February 2016,510000.0,3,2.00,1680.0,8080.0,1.0,No,NaN,...,8,1680.0,0,31,0,98074.0,47.6168,-122.045,1800.0,7503


In [3]:
y = data['Sale Price']
X = data.drop(['Sale Price'],axis = 1)
X.isna().sum()
X = data.drop(['No of Times Visited'],axis = 1)
X = X.drop(["Renovated Year"],axis = 1)
X.isna().sum()

ID                                           0
Date House was Sold                          0
Sale Price                                   4
No of Bedrooms                               0
No of Bathrooms                              4
Flat Area (in Sqft)                          9
Lot Area (in Sqft)                           9
No of Floors                                 0
Waterfront View                              0
Condition of the House                       0
Overall Grade                                0
Area of the House from Basement (in Sqft)    3
Basement Area (in Sqft)                      0
Age of House (in Years)                      0
Zipcode                                      1
Latitude                                     1
Longitude                                    1
Living Area after Renovation (in Sqft)       1
Lot Area after Renovation (in Sqft)          0
dtype: int64

In [4]:
def remove_outliers(data,col):
  q1 = data[col].quantile(0.25)
  q3 = data[col].quantile(0.75)
  iqr = q3 -q1
  lb = q1 - 1.5*iqr
  up = q3 + 1.5*iqr
  data[col] = data[col].clip(lower = lb,upper = up)
  return data[col]

numeric_columns = X.select_dtypes(include = 'number').columns.to_list()
numeric_columns.remove('Sale Price')
for col in numeric_columns:
    X[col] = remove_outliers(X,col)

In [5]:
y = pd.to_numeric(y, errors='coerce')
y = y.fillna(y.median)
y.isna().sum()


np.int64(0)

In [6]:
X  = X[numeric_columns]
X = X.apply(pd.to_numeric, errors='coerce')
for col in X.columns:
    X[col] = X[col].fillna(X[col].median())

print(X.dtypes)  

ID                                             int64
No of Bedrooms                               float64
No of Bathrooms                              float64
Flat Area (in Sqft)                          float64
Lot Area (in Sqft)                           float64
No of Floors                                 float64
Overall Grade                                float64
Area of the House from Basement (in Sqft)    float64
Basement Area (in Sqft)                        int64
Age of House (in Years)                        int64
Zipcode                                      float64
Latitude                                     float64
Longitude                                    float64
Living Area after Renovation (in Sqft)       float64
Lot Area after Renovation (in Sqft)          float64
dtype: object


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

In [56]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [57]:
print(y_train.head())
y_train = pd.to_numeric(y_train, errors="coerce")
y_test  = pd.to_numeric(y_test, errors="coerce")
print(type(y_train))
mask = ~y_train.isna()
X_train = X_train[mask]
y_train = y_train[mask]

mask_test = ~y_test.isna()
X_test = X_test[mask_test]
y_test = y_test[mask_test]

6325     325000.0
13473    257000.0
17614    228500.0
16970    288000.0
20868    479000.0
Name: Sale Price, dtype: object
<class 'pandas.core.series.Series'>


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

scaler_y = StandardScaler()

y_train_scaled = scaler_y.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_test_scaled  = scaler_y.transform(y_test.to_numpy().reshape(-1, 1))

In [59]:
X_train = torch.tensor(X_train,dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train_scaled, dtype=torch.float32)
y_test  = torch.tensor(y_test_scaled, dtype=torch.float32)

In [60]:
import numpy as np
print(np.isnan(X_train.numpy()).any(), np.isnan(y_train.numpy()).any())
print(np.isinf(X_train.numpy()).any(), np.isinf(y_train.numpy()).any())


False False
False False


In [61]:
from torch.utils.data import TensorDataset,DataLoader


In [64]:
train_dataset = TensorDataset(X_train,y_train)
test_dataset = TensorDataset(X_test,y_test)

train_loader = DataLoader(train_dataset,batch_size = 32,shuffle = True)
test_loader =DataLoader(test_dataset,batch_size = 32,shuffle = False)

In [65]:
import torch.nn as nn
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)


In [66]:
model = RegressionModel(X_train.shape[1])
criterion = nn.MSELoss()
optimizer =  torch.optim.Adam(model.parameters(),lr =0.001)

In [67]:

epochs = 50
for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  for X_batch,y_batch in train_loader: 
    optimizer.zero_grad()
    outputs = model(X_batch)
    y_batch = y_batch.view(-1, 1).float()
    loss = criterion(outputs,y_batch)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/50], Loss: 0.3601
Epoch [2/50], Loss: 0.2659
Epoch [3/50], Loss: 0.2378
Epoch [4/50], Loss: 0.2146
Epoch [5/50], Loss: 0.1977
Epoch [6/50], Loss: 0.1935
Epoch [7/50], Loss: 0.1864
Epoch [8/50], Loss: 0.1856
Epoch [9/50], Loss: 0.1780
Epoch [10/50], Loss: 0.1787
Epoch [11/50], Loss: 0.1718
Epoch [12/50], Loss: 0.1691
Epoch [13/50], Loss: 0.1662
Epoch [14/50], Loss: 0.1640
Epoch [15/50], Loss: 0.1619
Epoch [16/50], Loss: 0.1595
Epoch [17/50], Loss: 0.1597
Epoch [18/50], Loss: 0.1619
Epoch [19/50], Loss: 0.1605
Epoch [20/50], Loss: 0.1563
Epoch [21/50], Loss: 0.1514
Epoch [22/50], Loss: 0.1479
Epoch [23/50], Loss: 0.1485
Epoch [24/50], Loss: 0.1445
Epoch [25/50], Loss: 0.1419
Epoch [26/50], Loss: 0.1459
Epoch [27/50], Loss: 0.1416
Epoch [28/50], Loss: 0.1420
Epoch [29/50], Loss: 0.1382
Epoch [30/50], Loss: 0.1386
Epoch [31/50], Loss: 0.1352
Epoch [32/50], Loss: 0.1345
Epoch [33/50], Loss: 0.1360
Epoch [34/50], Loss: 0.1339
Epoch [35/50], Loss: 0.1331
Epoch [36/50], Loss: 0.1325
E

In [ ]:

model.eval()
val_loss = 0.0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        y_batch = y_batch.view(-1, 1).float()
        loss = criterion(outputs, y_batch)
        val_loss += loss.item()
    
val_loss /= len(test_loader)

from sklearn.metrics import r2_score
import numpy as np
model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test).numpy()  
    y_test_scaled = y_test.numpy()          

y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_test_scaled)

r2 = r2_score(y_true, y_pred)

print(f"Mse loss using criterion:{val_loss:.4f}")
print(f"Test R² Score: {r2:.4f}")


Mse loss using criterion:0.2028
Test R² Score: 0.8235


In [77]:
torch.save(model.state_dict(),"model_weights.pth")


In [78]:
model.load_state_dict(torch.load('model_weights.pth',weights_only=True))
model.eval()

RegressionModel(
  (net): Sequential(
    (0): Linear(in_features=15, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)